In [ ]:
import pandas as pd
from collections import defaultdict

In [ ]:
df_cpu_utilization = pd.read_csv('/content/Database_CPU_Utilization-2023_09_19_12_00_00-2023_10_17_11_00_00-UTC.csv', sep=',', skiprows=[0, 1], header=2)
df_read_iops = pd.read_csv('/content/Database_Read_IOPS-2023_09_19_12_00_00-2023_10_17_11_00_00-UTC.csv', sep=',', skiprows=[0, 1], header=2)
df_prod_latency = pd.read_csv('/content/Database_Prod_Latency-2023_09_19_12_00_00-2023_10_17_11_00_00-UTC.csv', sep=',', skiprows=[0, 1], header=2)
df_sessions_production = pd.read_csv('/content/Database_Sessions-2023_09_19_12_00_00-2023_10_17_11_00_00-UTC.csv', sep=',', skiprows=[0, 1], header=2)
df_elastic_beanstalk_cpu = pd.read_csv('/content/Elastic_Beanstalk_-_CPU_Usage-2023_09_19_12_00_00-2023_10_17_11_00_00-UTC.csv', sep=',', skiprows=[0, 1], header=2)
#df_load_balancer_requests = pd.read_csv('/content/Load_balancer_Total_requests_-_per_minute-2023_10_05_14_00_00-2023_10_05_16_00_00-UTC.csv', sep=',', skiprows=[0, 1], header=2)

# Renomear a segunda coluna em cada DataFrame
df_cpu_utilization.rename(columns={df_cpu_utilization.columns[2]: f'{df_cpu_utilization.columns[2]}1'}, inplace=True)
df_read_iops.rename(columns={df_read_iops.columns[2]: f'{df_read_iops.columns[2]}1'}, inplace=True)
df_prod_latency.rename(columns={df_prod_latency.columns[3]: f'{df_prod_latency.columns[3]}1'}, inplace=True)
df_sessions_production.rename(columns={df_sessions_production.columns[2]: f'{df_sessions_production.columns[2]}1'}, inplace=True)
df_elastic_beanstalk_cpu.rename(columns={df_elastic_beanstalk_cpu.columns[2]: f'{df_elastic_beanstalk_cpu.columns[2]}1'}, inplace=True)
#df_load_balancer_requests.rename(columns={df_load_balancer_requests.columns[1]: 'Request'}, inplace=True)

In [ ]:
def convert_columns(df):
    for column in df.columns:
        if column == 'Label':
            # Converte a coluna 'label' para o tipo de dados de data
            df[column] = pd.to_datetime(df[column])
        else:
            # Converte todas as outras colunas para inteiros
            df[column] = pd.to_numeric(df[column], errors='coerce', downcast='integer')

convert_columns(df_cpu_utilization)
convert_columns(df_read_iops)
convert_columns(df_prod_latency)
convert_columns(df_sessions_production)
convert_columns(df_elastic_beanstalk_cpu)
#convert_columns(df_load_balancer_requests)

In [ ]:
# Condições- USAR LINHAS AZUIS
condition_1 = df_sessions_production['DBLoad'] > 100
condition_2 = df_sessions_production['DBLoad.11'] > 50
condition_3 = df_cpu_utilization['CPUUtilization.11'] > 95

# Calcula a média das últimas 3 horas para a coluna 'ReadIOPS.11'
rolling_mean_read_iops = df_read_iops['ReadIOPS'].rolling(window=3).mean()
condition_4 = df_read_iops['ReadIOPS'] > (0.7 * rolling_mean_read_iops)

condition_5 = df_prod_latency['ReadLatency'] > 2
condition_6 = df_elastic_beanstalk_cpu['CPUUtilization'] > 80

# Lista para armazenar informações sobre as ocorrências de problemas
problemas = []

# Verifica as condições e registra as ocorrências
for index, row in df_sessions_production.iterrows():
    if condition_1[index]:
        problemas.append(f"Database Sessions de Production > 100. Horário: {row['Label'].strftime('%Y-%m-%d %H:%M:%S')}")
    if condition_2[index]:
        problemas.append(f"Database Sessions de Read Replica > 50. Horário: {row['Label'].strftime('%Y-%m-%d %H:%M:%S')}")
    if condition_3[index]:
        problemas.append(f"Database CPU Utilization > 95%. Horário: {row['Label'].strftime('%Y-%m-%d %H:%M:%S')}")
    if condition_4[index]:
        problemas.append(f"Database Read IOPS > 70% da média das últimas 3 horas. Horário: {row['Label'].strftime('%Y-%m-%d %H:%M:%S')}")
    if condition_5[index]:
        problemas.append(f"Database Prod Latency > 2 seg. Horário: {row['Label'].strftime('%Y-%m-%d %H:%M:%S')}")
    if condition_6[index]:
        problemas.append(f"Elastic Beanstalk - CPU Usage > 80%. Horário: {row['Label'].strftime('%Y-%m-%d %H:%M:%S')}")

# Exibe as informações sobre as ocorrências de problemas
if problemas:
    for problema in problemas:
        print(f"Pode haver um problema: {problema}")



In [ ]:
# Conjunto para rastrear os horários únicos com problemas
horarios_com_problemas = set()

# Verifica as condições e registra os horários únicos com problemas
for index, row in df_sessions_production.iterrows():
    if condition_1[index]:
        horarios_com_problemas.add(row['Label'].strftime('%Y-%m-%d %H:%M:%S'))
    if condition_2[index]:
        horarios_com_problemas.add(row['Label'].strftime('%Y-%m-%d %H:%M:%S'))
    if condition_3[index]:
        horarios_com_problemas.add(row['Label'].strftime('%Y-%m-%d %H:%M:%S'))
    if condition_4[index]:
        horarios_com_problemas.add(row['Label'].strftime('%Y-%m-%d %H:%M:%S'))
    if condition_5[index]:
        horarios_com_problemas.add(row['Label'].strftime('%Y-%m-%d %H:%M:%S'))
    if condition_6[index]:
        horarios_com_problemas.add(row['Label'].strftime('%Y-%m-%d %H:%M:%S'))

# Calcula o total de horas com problemas
total_horas_com_problemas = len(horarios_com_problemas)

# Exibe o total de horas com problemas
print(f"Total de horas com problemas: {total_horas_com_problemas} horas")

In [ ]:
# Dicionário para rastrear o número de horas com problemas por mês
horas_com_problemas_por_mes = defaultdict(int)

# Verifica as condições e registra o número de horas com problemas por mês
for index, row in df_sessions_production.iterrows():
    mes = row['Label'].strftime('%Y-%m')
    if condition_1[index] or condition_2[index] or condition_3[index] or condition_4[index] or condition_5[index] or condition_6[index]:
        horas_com_problemas_por_mes[mes] += 1

# Calcula a média das horas com problemas para um mês específico (por exemplo, setembro - 2023-09)
mes_especifico = '2023-09'
total_horas_no_mes = horas_com_problemas_por_mes[mes_especifico]
media_horas_no_mes = total_horas_no_mes / 30  # Assumindo 30 dias no mês

# Exibe a média de horas com problemas no mês específico
print(f"Média de horas com problemas no mês: {media_horas_no_mes:.2f} horas")

Média de horas com problemas no mês: 5.53 horas
